<a href="https://colab.research.google.com/github/Method-for-Software-System-Development/Cloud_Computing/blob/tirgul-Omer-Matan/Lab5_Omer_Matan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install firebase
!pip install altair vega_datasets

In [8]:
from firebase import firebase
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import altair as alt

FBconn = firebase.FirebaseApplication('https://lab5-cloudcomputing-default-rtdb.firebaseio.com/', None)

In [11]:
def add_single_word(word):
    """
    Add a single word to the database.
    If the word already exists, increment its count by 1.
    """
    word = word.lower()
    existing = FBconn.get('/words/', word)
    if existing:
        FBconn.put('/words/', word, existing + 1)
    else:
        FBconn.put('/words/', word, 1)

def add_text(text):
    """
    Add multiple words from a given text to the database.
    Each word is processed individually.
    """
    words = text.lower().split()
    for word in words:
        add_single_word(word)

def update_word_count(word, new_count):
    """
    Manually update the occurrence count of a specific word.
    """
    word = word.lower()
    FBconn.put('/words/', word, new_count)

def delete_word(word):
    """
    Delete a word and its count from the database.
    """
    word = word.lower()
    FBconn.delete('/words/', word)

def get_all_words():
    return FBconn.get('/words/', None)

def display_words():
    """
    Display all words stored in the database along with their counts,
    and plot a bar chart of word frequencies.
    """
    all_words = get_all_words()
    if all_words:
        for word, count in all_words.items():
            print(f"{word} : {count}")

        # Create a dataframe for plotting
        df = pd.DataFrame(list(all_words.items()), columns=["Word", "Count"])

        # Plot using Altair
        chart = alt.Chart(df).mark_bar().encode(
            x=alt.X('Word:N', sort='-y'),  # Sort descending by count
            y='Count:Q',
            color='Word:N'
        ).properties(
            title='Word Frequencies'
        )
        display(chart)

    else:
        print("No words found.")

In [26]:
# Widgets:

# Create buttons for each action
btn_add_word = widgets.Button(description="Add Single Word")
btn_add_text = widgets.Button(description="Add Full Text")
btn_update_word = widgets.Button(description="Update Word Count")
btn_delete_word = widgets.Button(description="Delete Word")
btn_view_words = widgets.Button(description="View All Words")
btn_exit = widgets.Button(description="Exit Menu")

# Create input widgets
word_input = widgets.Text(description="Word:")
text_input = widgets.Text(description="Text:")
update_word_input = widgets.Text(description="Word to Update:")
update_count_input = widgets.IntText(description="New Count:")
delete_word_input = widgets.Text(description="Word to Delete:")

# Create output area
output = widgets.Output()

# Functions for Button Clicks:

def on_add_word_clicked(b):
    with output:
        clear_output()
        add_single_word(word_input.value)
        print(f"Added/Updated word: {word_input.value}")

def on_add_text_clicked(b):
    with output:
        clear_output()
        add_text(text_input.value)
        print("Text processed and words updated!")

def on_update_word_clicked(b):
    with output:
        clear_output()
        update_word_count(update_word_input.value, update_count_input.value)
        print(f"Updated word '{update_word_input.value}' to {update_count_input.value} appearances.")

def on_delete_word_clicked(b):
    with output:
        clear_output()
        delete_word(delete_word_input.value)
        print(f"Deleted word: {delete_word_input.value}")

def on_view_words_clicked(b):
    with output:
        clear_output()
        display_words()

def on_exit_clicked(b):
    with output:
        clear_output()
        print("Exited menu.")

# Attach event handlers to buttons
btn_add_word.on_click(on_add_word_clicked)
btn_add_text.on_click(on_add_text_clicked)
btn_update_word.on_click(on_update_word_clicked)
btn_delete_word.on_click(on_delete_word_clicked)
btn_view_words.on_click(on_view_words_clicked)
btn_exit.on_click(on_exit_clicked)

# Layout:

menu = widgets.VBox(
    [
        widgets.HTML("<h2 style='color: teal; text-align:left;'>Word Tracker Menu</h2>"),

        widgets.HBox([word_input, btn_add_word],
                     layout=widgets.Layout(justify_content='flex-start')),

        widgets.HBox([text_input, btn_add_text],
                     layout=widgets.Layout(justify_content='flex-start')),

        widgets.HBox([update_word_input, update_count_input, btn_update_word],
                     layout=widgets.Layout(justify_content='flex-start')),

        widgets.HBox([delete_word_input, btn_delete_word],
                     layout=widgets.Layout(justify_content='flex-start')),

        btn_view_words,
        btn_exit,
        output
    ],
    layout=widgets.Layout(
        border='2px solid lightgray',
        padding='20px',
        align_items='flex-start',
        width='100%',
        margin='0 auto',
        background_color='whitesmoke'
    )
)

# Display the menu
display(menu)